In [31]:
# 2D CNN
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

# 2D CNN 모델 정의
class Simple2DCNN(nn.Module):
    def __init__(self):
        super(Simple2DCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 160 * 160, 512)  # 640x640 크기에 맞게 조정
        self.fc2 = nn.Linear(512, 2)  # 예시: 클래스가 2개 (변화 감지, 비변화)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 64 * 160 * 160)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def extract_objects(results, confidence_threshold):
    objects = []
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = r
        if score > confidence_threshold:
            objects.append({
                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                'score': score,
                'class_id': int(class_id)
            })
    return objects

def compare_objects(objects1, objects2, iou_threshold):
    added = []
    removed = []
    
    matched1 = set()
    matched2 = set()
    
    for i, obj1 in enumerate(objects1):
        for j, obj2 in enumerate(objects2):
            if i in matched1 or j in matched2:
                continue
            
            iou = calculate_iou(obj1['bbox'], obj2['bbox'])
            if iou > iou_threshold:
                matched1.add(i)
                matched2.add(j)
                break
        
        if i not in matched1:
            removed.append(obj1)
    
    for j, obj2 in enumerate(objects2):
        if j not in matched2:
            added.append(obj2)
    
    return added, removed

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou
    pass    

def visualize_changes(image1, image2, added, removed):
    result_image = np.hstack((image1, image2))
    height, width = image1.shape[:2]
    
    for obj in added:
        x1, y1, x2, y2 = obj['bbox']
        cv2.rectangle(result_image, (x1 + width, y1), (x2 + width, y2), (0, 255, 0), 2)
        cv2.putText(result_image, 'Added', (x1 + width, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    for obj in removed:
        x1, y1, x2, y2 = obj['bbox']
        cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(result_image, 'Removed', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    cv2.imshow('Changes Detected', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    pass

def analyze_results(objects1, objects2, added, removed):
    print(f"Image 1: {len(objects1)} objects detected")
    print(f"Image 2: {len(objects2)} objects detected")
    print(f"Added objects: {len(added)}")
    print(f"Removed objects: {len(removed)}")
    
    class_counts1 = {}
    class_counts2 = {}
    
    for obj in objects1:
        class_counts1[obj['class_id']] = class_counts1.get(obj['class_id'], 0) + 1
    
    for obj in objects2:
        class_counts2[obj['class_id']] = class_counts2.get(obj['class_id'], 0) + 1
    
    print("\nClass distribution in Image 1:")
    for class_id, count in class_counts1.items():
        print(f"Class {class_id}: {count}")
    
    print("\nClass distribution in Image 2:")
    for class_id, count in class_counts2.items():
        print(f"Class {class_id}: {count}")


# 변화 감지 함수
def detect_changes_2d(image1_path, image2_path, model_path, cnn_model, confidence_threshold=0.5, iou_threshold=0.5):
    # YOLO 모델 로드
    model = YOLO(model_path)

    # 이미지 로드
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # YOLO를 사용하여 객체 탐지
    results1 = model(image1)[0]
    results2 = model(image2)[0]

    # 객체 추출
    objects1 = extract_objects(results1, confidence_threshold)
    objects2 = extract_objects(results2, confidence_threshold)

    # 객체 비교 (기존 방식 유지)
    added, removed = compare_objects(objects1, objects2, iou_threshold)

    # 이미지를 2D CNN에 입력
    image1_2d = prepare_image_for_2d_cnn(image1)
    image2_2d = prepare_image_for_2d_cnn(image2)

    # 2D CNN을 통한 특징 추출 및 변화 감지
    features1 = cnn_model(image1_2d)
    features2 = cnn_model(image2_2d)
    
    difference = compute_difference_2d(features1, features2)

    # 결과 시각화
    visualize_changes(image1, image2, added, removed)

    # 결과 분석
    analyze_results(objects1, objects2, added, removed)

# 2D CNN에 입력할 이미지를 준비하는 함수
def prepare_image_for_2d_cnn(image):
    # 이미지를 2D CNN에 입력할 수 있도록 변환
    image = cv2.resize(image, (640, 640))
    image_2d = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
    return torch.Tensor(image_2d).unsqueeze(0)  # (B, C, H, W)

# 2D CNN을 통한 특징 비교 함수
def compute_difference_2d(features1, features2):
    # 간단한 유클리드 거리 계산
    return torch.norm(features1 - features2, p=2).item()

# 나머지 기존 함수들 유지 (extract_objects, compare_objects, calculate_iou, visualize_changes, analyze_results)

# CNN 모델 초기화
cnn_model = Simple2DCNN()

# 사용 예시
image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 1.jpg'
image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 2.jpg'
model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'

detect_changes_2d(image1_path, image2_path, model_path, cnn_model)



0: 640x640 3 crossarms, 1 polo, 9 wires, 30.0ms
Speed: 3.0ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 crossarms, 1 polo, 12 wires, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Image 1: 12 objects detected
Image 2: 14 objects detected
Added objects: 2
Removed objects: 0

Class distribution in Image 1:
Class 2: 8
Class 0: 3
Class 1: 1

Class distribution in Image 2:
Class 2: 10
Class 0: 3
Class 1: 1


In [32]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Simple2DCNN(nn.Module):
    def __init__(self):
        super(Simple2DCNN, self).__init__()
        # 더 깊은 CNN 구조로 변경
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 80 * 80, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(torch.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(torch.relu(self.batch_norm3(self.conv3(x))))
        x = x.view(-1, 256 * 80 * 80)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

def extract_objects(results, confidence_threshold):
    objects = []
    # 신뢰도 점수에 따른 정렬 추가
    boxes = sorted(results.boxes.data.tolist(), key=lambda x: x[4], reverse=True)
    
    for r in boxes:
        x1, y1, x2, y2, score, class_id = r
        if score > confidence_threshold:
            # 바운딩 박스 정규화 추가
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            w, h = x2 - x1, y2 - y1
            if w > 0 and h > 0:  # 유효한 바운딩 박스만 추가
                objects.append({
                    'bbox': [x1, y1, x2, y2],
                    'score': score,
                    'class_id': int(class_id),
                    'area': w * h  # 면적 정보 추가
                })
    return objects

def compare_objects(objects1, objects2, iou_threshold):
    added = []
    removed = []
    matched1 = set()
    matched2 = set()
    
    # 면적 기반 정렬 추가
    objects1 = sorted(objects1, key=lambda x: x['area'], reverse=True)
    objects2 = sorted(objects2, key=lambda x: x['area'], reverse=True)
    
    for i, obj1 in enumerate(objects1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(objects2):
            if j in matched2:
                continue
                
            iou = calculate_iou(obj1['bbox'], obj2['bbox'])
            # 클래스 ID가 같은 경우에만 매칭
            if iou > best_iou and obj1['class_id'] == obj2['class_id']:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            matched1.add(i)
            matched2.add(best_match)
        else:
            removed.append(obj1)
    
    for j, obj2 in enumerate(objects2):
        if j not in matched2:
            added.append(obj2)
    
    return added, removed

def detect_changes_2d(image1_path, image2_path, model_path, cnn_model, 
                     confidence_threshold=0.45,  # 신뢰도 임계값 조정
                     iou_threshold=0.4):        # IOU 임계값 조정
    # YOLO 모델 로드 및 설정
    model = YOLO(model_path)
    model.conf = confidence_threshold  # 모델 신뢰도 임계값 설정
    model.iou = iou_threshold         # 모델 IOU 임계값 설정
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 정규화
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO 예측
    results1 = model(image1, verbose=False)[0]  # verbose=False로 불필요한 출력 제거
    results2 = model(image2, verbose=False)[0]
    
    # 객체 추출 및 비교
    objects1 = extract_objects(results1, confidence_threshold)
    objects2 = extract_objects(results2, confidence_threshold)
    
    # 변화 감지
    added, removed = compare_objects(objects1, objects2, iou_threshold)
    
    # CNN 특징 추출
    with torch.no_grad():  # 추론 모드로 변경
        image1_2d = prepare_image_for_2d_cnn(image1)
        image2_2d = prepare_image_for_2d_cnn(image2)
        features1 = cnn_model(image1_2d)
        features2 = cnn_model(image2_2d)
    
    # 결과 시각화 및 분석
    visualize_changes(image1, image2, added, removed)
    analyze_results(objects1, objects2, added, removed)
    
    return added, removed

def prepare_image_for_2d_cnn(image):
    # 이미지 정규화 추가
    image = image.astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))
    image = torch.Tensor(image).unsqueeze(0)
    return image

# 사용 예시
if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 7.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 8.jpg'
    model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    # CNN 모델 초기화 및 평가 모드로 설정
    cnn_model = Simple2DCNN()
    cnn_model.eval()
    
    # 변화 감지 실행
    detect_changes_2d(image1_path, image2_path, model_path, cnn_model)

Image 1: 11 objects detected
Image 2: 15 objects detected
Added objects: 4
Removed objects: 0

Class distribution in Image 1:
Class 0: 2
Class 2: 8
Class 1: 1

Class distribution in Image 2:
Class 0: 2
Class 2: 12
Class 1: 1


In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Simple2DCNN(nn.Module):
    def __init__(self, input_channels=3):
        super(Simple2DCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 160 * 160, 512)
        self.fc2 = nn.Linear(512, 2)  # 변화 있음/없음 분류

    def forward(self, x):
        x = self.pool(torch.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(torch.relu(self.batch_norm2(self.conv2(x))))
        x = x.view(-1, 128 * 160 * 160)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def create_detection_map(image_shape, detections, confidence_threshold=0.25):
    """YOLO 검출 결과를 히트맵으로 변환"""
    detection_map = np.zeros(image_shape[:2], dtype=np.float32)
    
    for det in detections.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = det
        if conf > confidence_threshold:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            detection_map[y1:y2, x1:x2] = conf
    
    return detection_map

def detect_changes(image1_path, image2_path, yolo_model_path, cnn_model=None):
    # YOLO 모델 로드
    yolo = YOLO(yolo_model_path)
    
    # 이미지 로드 및 크기 조정
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_s    ize)
    
    # YOLO로 객체 검출
    results1 = yolo(image1)[0]
    results2 = yolo(image2)[0]
    
    # 검출 결과를 히트맵으로 변환
    detection_map1 = create_detection_map(image1.shape, results1)
    detection_map2 = create_detection_map(image2.shape, results2)
    
    # 검출 맵을 3채널로 확장 (CNN 입력용)
    detection_maps = np.stack([
        detection_map1,
        detection_map2,
        np.abs(detection_map1 - detection_map2)
    ], axis=2)
    
    # 결과 시각화
    visualize_results(image1, image2, detection_map1, detection_map2, results1, results2)
    
    # CNN 모델이 제공된 경우 변화 분류 수행
    if cnn_model is not None:
        with torch.no_grad():
            # 입력 데이터 준비
            input_tensor = torch.FloatTensor(detection_maps).permute(2, 0, 1).unsqueeze(0)
            
            # 변화 예측
            prediction = cnn_model(input_tensor)
            probability = torch.softmax(prediction, dim=1)
            change_prob = probability[0][1].item()
            
            print(f"Change probability: {change_prob:.2%}")
            return change_prob > 0.5
    
    return None

def visualize_results(image1, image2, map1, map2, yolo_results1, yolo_results2):
    # YOLO 검출 결과 시각화
    det_image1 = image1.copy()
    det_image2 = image2.copy()
    
    # YOLO 바운딩 박스 그리기
    for result, img in [(yolo_results1, det_image1), (yolo_results2, det_image2)]:
        for box in result.boxes.data.tolist():
            x1, y1, x2, y2, conf, cls = box
            if conf > 0.25:  # 신뢰도 임계값
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f'{conf:.2f}', (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # 히트맵 시각화
    map1_colored = cv2.applyColorMap((map1 * 255).astype(np.uint8), cv2.COLORMAP_JET)
    map2_colored = cv2.applyColorMap((map2 * 255).astype(np.uint8), cv2.COLORMAP_JET)
    
    # 결과 합치기
    top_row = np.hstack((det_image1, det_image2))
    bottom_row = np.hstack((map1_colored, map2_colored))
    combined = np.vstack((top_row, bottom_row))
    
    # 창 크기 조정
    scale = 0.7
    display_size = (int(combined.shape[1] * scale), int(combined.shape[0] * scale))
    combined = cv2.resize(combined, display_size)
    
    cv2.imshow('Detection Results', combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 5.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 6.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    # 2D CNN 모델 초기화 (선택적)
    cnn_model = Simple2DCNN(input_channels=3)
    
    # 변화 감지 실행
    result = detect_changes(image1_path, image2_path, yolo_model_path, cnn_model)
    
    if result is not None:
        print(f"Change detected: {result}")


0: 640x640 4 crossarms, 1 polo, 15 wires, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 crossarms, 1 polo, 19 wires, 12.0ms
Speed: 3.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Change probability: 55.55%
Change detected: True


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

def calculate_iou(box1, box2):
    """두 바운딩 박스 간의 IoU 계산"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection + 1e-6)
    return iou

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25, iou_threshold=0.5):
    # YOLO 모델 로드
    model = YOLO(yolo_model_path)
    
    # 이미지 로드
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출
    results1 = model(image1)[0]
    results2 = model(image2)[0]
    
    # 검출 결과 추출
    boxes1 = []
    boxes2 = []
    
    for r in results1.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes1.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    for r in results2.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes2.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    # 객체 매칭 및 변화 감지
    disappeared = []  # 사라진 객체
    appeared = []    # 새로 나타난 객체
    
    # 같은 객체 매칭
    for i, obj1 in enumerate(boxes1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(boxes2):
            if obj2['matched'] or obj1['class'] != obj2['class']:
                continue
                
            iou = calculate_iou(obj1['box'], obj2['box'])
            if iou > best_iou:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            boxes1[i]['matched'] = True
            boxes2[best_match]['matched'] = True
        else:
            disappeared.append(obj1)
    
    # 새로 나타난 객체 찾기
    for obj2 in boxes2:
        if not obj2['matched']:
            appeared.append(obj2)
    
    # 결과 시각화
    def create_visualization(image1, image2, disappeared, appeared):
        # 원본 이미지 복사
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체 표시 (빨간색)
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img1, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(vis_img1, f'Disappeared (conf: {obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체 표시 (초록색)
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis_img2, f'Appeared (conf: {obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 히트맵 생성
        heatmap1 = np.zeros(image1.shape[:2], dtype=np.float32)
        heatmap2 = np.zeros(image2.shape[:2], dtype=np.float32)
        
        # 사라진 객체 히트맵
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            heatmap1[y1:y2, x1:x2] = obj['conf']
        
        # 새로 나타난 객체 히트맵
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            heatmap2[y1:y2, x1:x2] = obj['conf']
        
        # 히트맵 컬러 변환
        heatmap1_colored = cv2.applyColorMap((heatmap1 * 255).astype(np.uint8), cv2.COLORMAP_JET)
        heatmap2_colored = cv2.applyColorMap((heatmap2 * 255).astype(np.uint8), cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, heatmap1_colored, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, heatmap2_colored, alpha, 0)
        
        # 최종 결과 이미지 생성
        result = np.hstack((overlay1, overlay2))
        
        return result, heatmap1_colored, heatmap2_colored
    
    # 시각화 실행
    result_image, hmap1, hmap2 = create_visualization(image1, image2, disappeared, appeared)
    
    # 결과 출력
    cv2.imshow('Object Changes', result_image)
    
    # 분석 결과 출력
    print("\nChange Detection Results:")
    print(f"Total objects in first image: {len(boxes1)}")
    print(f"Total objects in second image: {len(boxes2)}")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

# 실행 코드
if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 3.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 4.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    disappeared, appeared = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25,  # 신뢰도 임계값
        iou_threshold=0.5     # IoU 임계값
    )


0: 640x640 1 crossarm, 1 polo, 13 wires, 13.6ms
Speed: 4.0ms preprocess, 13.6ms inference, 86.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crossarm, 1 polo, 15 wires, 11.4ms
Speed: 2.0ms preprocess, 11.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Total objects in first image: 15
Total objects in second image: 17
Disappeared objects: 0
Appeared objects: 2

Appeared objects details:
Class: 2, Confidence: 0.94
Class: 2, Confidence: 0.93


In [4]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Conv3DNet(nn.Module):
    def __init__(self, in_channels=3):
        super(Conv3DNet, self).__init__()
        self.features = nn.Sequential(
            # First 3D Conv layer
            nn.Conv3d(in_channels, 32, kernel_size=(2, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            
            # Second 3D Conv layer
            nn.Conv3d(32, 16, kernel_size=(1, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(16),
            
            # Final layer for change detection
            nn.Conv3d(16, 1, kernel_size=(1, 1, 1)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(2)  # temporal dimension 제거
        return x

def calculate_iou(box1, box2):
    """두 바운딩 박스 간의 IoU 계산"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection + 1e-6)
    return iou

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25, iou_threshold=0.5):
    # YOLO 모델 로드
    yolo_model = YOLO(yolo_model_path)
    
    # 3D CNN 모델 초기화
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    temporal_model = Conv3DNet().to(device)
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출
    results1 = yolo_model(image1)[0]
    results2 = yolo_model(image2)[0]
    
    # 3D CNN을 위한 입력 준비
    imgs_tensor = torch.from_numpy(np.stack([image1, image2]))
    imgs_tensor = imgs_tensor.permute(3, 0, 1, 2).unsqueeze(0).float() / 255.0
    imgs_tensor = imgs_tensor.to(device)
    
    # 3D CNN으로 변화 감지
    with torch.no_grad():
        change_score = temporal_model(imgs_tensor)
        change_score = change_score.squeeze()
    
    # 검출 결과 추출
    boxes1 = []
    boxes2 = []
    
    for r in results1.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes1.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    for r in results2.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes2.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    # 객체 매칭 및 변화 감지
    disappeared = []
    appeared = []
    modified = []
    
    for i, obj1 in enumerate(boxes1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(boxes2):
            if obj2['matched'] or obj1['class'] != obj2['class']:
                continue
                
            iou = calculate_iou(obj1['box'], obj2['box'])
            if iou > best_iou:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            boxes1[i]['matched'] = True
            boxes2[best_match]['matched'] = True
            
            # 변화 스코어를 확인하여 수정된 객체 감지
            x1, y1, x2, y2 = obj1['box']
            change_region = change_score[y1:y2, x1:x2].mean().item()
            if change_region > 0.3:
                modified.append(obj1)
        else:
            disappeared.append(obj1)
    
    # 새로 나타난 객체 찾기
    for obj2 in boxes2:
        if not obj2['matched']:
            appeared.append(obj2)
    
    # 결과 시각화
    def create_visualization(image1, image2, disappeared, appeared, modified, change_score):
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체 표시 (빨간색)
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img1, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(vis_img1, f'Disappeared ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체 표시 (초록색)
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis_img2, f'Appeared ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 수정된 객체 표시 (파란색)
        for obj in modified:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img1, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.rectangle(vis_img2, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(vis_img1, f'Modified ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(vis_img2, f'Modified ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # 변화 히트맵 생성
        change_heatmap = change_score.cpu().numpy()
        change_heatmap = (change_heatmap * 255).astype(np.uint8)
        change_heatmap = cv2.applyColorMap(change_heatmap, cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, change_heatmap, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, change_heatmap, alpha, 0)
        
        result = np.hstack((overlay1, overlay2))
        return result
    
    # 시각화 실행
    result_image = create_visualization(image1, image2, disappeared, appeared, modified, change_score)
    
    # 결과 출력
    cv2.imshow('Object Changes (3D CNN)', result_image)
    
    print("\nChange Detection Results:")
    print(f"Total objects in first image: {len(boxes1)}")
    print(f"Total objects in second image: {len(boxes2)}")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    print(f"Modified objects: {len(modified)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
            
    if modified:
        print("\nModified objects details:")
        for obj in modified:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared, modified

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 7.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 8.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    disappeared, appeared, modified = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25,
        iou_threshold=0.5
    )


0: 640x640 2 crossarms, 1 polo, 10 wires, 16.7ms
Speed: 3.0ms preprocess, 16.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 crossarms, 1 polo, 13 wires, 17.3ms
Speed: 2.0ms preprocess, 17.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Total objects in first image: 13
Total objects in second image: 16
Disappeared objects: 0
Appeared objects: 3
Modified objects: 13

Appeared objects details:
Class: 2, Confidence: 0.65
Class: 2, Confidence: 0.57
Class: 2, Confidence: 0.40

Modified objects details:
Class: 0, Confidence: 0.86
Class: 0, Confidence: 0.86
Class: 2, Confidence: 0.83
Class: 2, Confidence: 0.79
Class: 2, Confidence: 0.79
Class: 2, Confidence: 0.76
Class: 2, Confidence: 0.76
Class: 2, Confidence: 0.66
Class: 2, Confidence: 0.53
Class: 2, Confidence: 0.52
Class: 1, Confidence: 0.51
Class: 2, Confidence: 0.40
Class: 2, Confidence: 0.32


In [8]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Conv3DNet(nn.Module):
    def __init__(self, in_channels=3):
        super(Conv3DNet, self).__init__()
        self.features = nn.Sequential(
            # First 3D Conv layer
            nn.Conv3d(in_channels, 32, kernel_size=(2, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            
            # Second 3D Conv layer
            nn.Conv3d(32, 16, kernel_size=(1, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(16),
            
            # Final layer for change detection
            nn.Conv3d(16, 1, kernel_size=(1, 1, 1)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(2)  # temporal dimension 제거
        return x

def calculate_iou(box1, box2):
    """두 바운딩 박스 간의 IoU 계산"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection + 1e-6)
    return iou

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25, iou_threshold=0.5):
    # YOLO 모델 로드
    yolo_model = YOLO(yolo_model_path)
    
    # 3D CNN 모델 초기화
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    temporal_model = Conv3DNet().to(device)
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출
    results1 = yolo_model(image1)[0]
    results2 = yolo_model(image2)[0]
    
    # 3D CNN을 위한 입력 준비
    imgs_tensor = torch.from_numpy(np.stack([image1, image2]))
    imgs_tensor = imgs_tensor.permute(3, 0, 1, 2).unsqueeze(0).float() / 255.0
    imgs_tensor = imgs_tensor.to(device)
    
    # 3D CNN으로 변화 감지
    with torch.no_grad():
        change_score = temporal_model(imgs_tensor)
        change_score = change_score.squeeze()
    
    # 검출 결과 추출
    boxes1 = []
    boxes2 = []
    
    for r in results1.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes1.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    for r in results2.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = r
        if conf > conf_threshold:
            boxes2.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'conf': conf,
                'class': int(cls),
                'matched': False
            })
    
    # 객체 매칭 및 변화 감지
    disappeared = []
    appeared = []
    
    for i, obj1 in enumerate(boxes1):
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(boxes2):
            if obj2['matched'] or obj1['class'] != obj2['class']:
                continue
                
            iou = calculate_iou(obj1['box'], obj2['box'])
            if iou > best_iou:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            boxes1[i]['matched'] = True
            boxes2[best_match]['matched'] = True
        else:
            disappeared.append(obj1)
    
    # 새로 나타난 객체 찾기
    for obj2 in boxes2:
        if not obj2['matched']:
            appeared.append(obj2)
    
    # 결과 시각화
    def create_visualization(image1, image2, disappeared, appeared, change_score):
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체 표시 (빨간색)
        for obj in disappeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img1, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(vis_img1, f'Disappeared ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체 표시 (초록색)
        for obj in appeared:
            x1, y1, x2, y2 = obj['box']
            cv2.rectangle(vis_img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis_img2, f'Appeared ({obj["conf"]:.2f})', 
                       (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 변화 히트맵 생성
        change_heatmap = change_score.cpu().numpy()
        change_heatmap = (change_heatmap * 255).astype(np.uint8)
        change_heatmap = cv2.applyColorMap(change_heatmap, cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, change_heatmap, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, change_heatmap, alpha, 0)
        
        result = np.hstack((overlay1, overlay2))
        return result
    
    # 시각화 실행
    result_image = create_visualization(image1, image2, disappeared, appeared, change_score)
    
    # 결과 출력
    cv2.imshow('Object Changes (3D CNN)', result_image)
    
    print("\nChange Detection Results:")
    print(f"Total objects in first image: {len(boxes1)}")
    print(f"Total objects in second image: {len(boxes2)}")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 7.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 8.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    disappeared, appeared = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25,
        iou_threshold=0.5
    )


0: 640x640 2 crossarms, 1 polo, 10 wires, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 crossarms, 1 polo, 13 wires, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Total objects in first image: 13
Total objects in second image: 16
Disappeared objects: 0
Appeared objects: 3

Appeared objects details:
Class: 2, Confidence: 0.65
Class: 2, Confidence: 0.57
Class: 2, Confidence: 0.40


In [21]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Conv3DNet(nn.Module):
    def __init__(self, in_channels=3):
        super(Conv3DNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(in_channels, 32, kernel_size=(2, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            
            nn.Conv3d(32, 16, kernel_size=(1, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(16),
            
            nn.Conv3d(16, 1, kernel_size=(1, 1, 1)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(2)
        return x

def calculate_mask_iou(mask1, mask2):
    """두 마스크 간의 IoU 계산"""
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / (union + 1e-6)

def calculate_distance(mask1, mask2):
    """두 마스크의 중심점 간 거리 계산"""
    y1, x1 = np.where(mask1)
    y2, x2 = np.where(mask2)
    
    center1 = np.array([np.mean(y1), np.mean(x1)])
    center2 = np.array([np.mean(y2), np.mean(x2)])
    
    return np.linalg.norm(center1 - center2)

def match_wires(masks1, masks2, distance_threshold=30):
    """전선 객체들을 매칭하는 함수"""
    wire_masks1 = [m for m in masks1 if m['class'] == 2]  # wire class
    wire_masks2 = [m for m in masks2 if m['class'] == 2]
    
    # 각 전선의 y좌표 평균값 계산
    for w1 in wire_masks1:
        w1['y_center'] = np.mean(np.where(w1['mask'])[0])
    for w2 in wire_masks2:
        w2['y_center'] = np.mean(np.where(w2['mask'])[0])
    
    # y좌표 기준으로 정렬
    wire_masks1.sort(key=lambda x: x['y_center'])
    wire_masks2.sort(key=lambda x: x['y_center'])
    
    # 순서대로 매칭
    disappeared = []
    appeared = []
    
    i, j = 0, 0
    while i < len(wire_masks1) and j < len(wire_masks2):
        w1 = wire_masks1[i]
        w2 = wire_masks2[j]
        
        y_diff = abs(w1['y_center'] - w2['y_center'])
        
        if y_diff < 20:  # y좌표 차이가 20픽셀 이내
            w1['matched'] = True
            w2['matched'] = True
            i += 1
            j += 1
        elif w1['y_center'] < w2['y_center']:
            disappeared.append(w1)
            i += 1
        else:
            appeared.append(w2)
            j += 1
    
    # 남은 전선들 처리
    while i < len(wire_masks1):
        disappeared.append(wire_masks1[i])
        i += 1
    
    while j < len(wire_masks2):
        appeared.append(wire_masks2[j])
        j += 1
    
    return disappeared, appeared

def preprocess_masks(masks):
    """마스크 전처리 함수"""
    processed = []
    for mask in masks:
        # 너무 작은 마스크 제거
        if np.sum(mask['mask']) < 50:  # 50픽셀 미만 제거
            continue
        
        # 전선 클래스의 경우 형태 검사
        if mask['class'] == 2:  # wire class
            h, w = mask['mask'].shape
            if h/w < 2:  # 세로/가로 비율 2 미만 제외
                continue
            
        processed.append(mask)
    return processed

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25, iou_threshold=0.5):
    # YOLO 모델 로드
    yolo_model = YOLO(yolo_model_path)
    
    # 3D CNN 모델 초기화
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    temporal_model = Conv3DNet().to(device)
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # YOLO로 객체 검출 및 세그멘테이션
    results1 = yolo_model(image1)[0]
    results2 = yolo_model(image2)[0]
    
    # 3D CNN을 위한 입력 준비
    imgs_tensor = torch.from_numpy(np.stack([image1, image2]))
    imgs_tensor = imgs_tensor.permute(3, 0, 1, 2).unsqueeze(0).float() / 255.0
    imgs_tensor = imgs_tensor.to(device)
    
    # 3D CNN으로 변화 감지
    with torch.no_grad():
        change_score = temporal_model(imgs_tensor)
        change_score = change_score.squeeze()
    
    # 마스크 추출 및 전처리
    masks1 = []
    masks2 = []
    
    # 첫 번째 이미지의 마스크 추출
    for i in range(len(results1.masks.data)):
        conf = results1.boxes.conf[i].item()
        if conf > conf_threshold:
            mask = results1.masks.data[i].cpu().numpy()
            cls = int(results1.boxes.cls[i].item())
            masks1.append({
                'mask': mask,
                'conf': conf,
                'class': cls,
                'matched': False
            })
    
    # 두 번째 이미지의 마스크 추출
    for i in range(len(results2.masks.data)):
        conf = results2.boxes.conf[i].item()
        if conf > conf_threshold:
            mask = results2.masks.data[i].cpu().numpy()
            cls = int(results2.boxes.cls[i].item())
            masks2.append({
                'mask': mask,
                'conf': conf,
                'class': cls,
                'matched': False
            })
    
    # 마스크 전처리
    masks1 = preprocess_masks(masks1)
    masks2 = preprocess_masks(masks2)
    
    # 객체 매칭 및 변화 감지
    disappeared = []
    appeared = []
    
    # 전선 객체 먼저 처리
    wire_disappeared, wire_appeared = match_wires(masks1, masks2)
    disappeared.extend(wire_disappeared)
    appeared.extend(wire_appeared)
    
    # 나머지 객체들 처리
    for i, obj1 in enumerate(masks1):
        if obj1['class'] == 2 or obj1['matched']:  # 전선이거나 이미 매칭된 객체는 건너뛰기
            continue
            
        best_iou = iou_threshold
        best_match = -1
        
        for j, obj2 in enumerate(masks2):
            if obj2['class'] == 2 or obj2['matched'] or obj1['class'] != obj2['class']:
                continue
                
            iou = calculate_mask_iou(obj1['mask'], obj2['mask'])
            if iou > best_iou:
                best_iou = iou
                best_match = j
        
        if best_match >= 0:
            masks1[i]['matched'] = True
            masks2[best_match]['matched'] = True
        else:
            disappeared.append(obj1)
    
    # 남은 새로운 객체들 처리
    for obj2 in masks2:
        if obj2['class'] != 2 and not obj2['matched']:  # 전선이 아니고 매칭되지 않은 객체
            appeared.append(obj2)

    def create_visualization(image1, image2, disappeared, appeared, change_score):
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체의 마스크 표시 (빨간색)
        for obj in disappeared:
            mask = obj['mask'].astype(bool)
            vis_img1[mask] = vis_img1[mask] * 0.7 + np.array([0, 0, 255]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img1, contours, -1, (0, 0, 255), 2)
            
            # 신뢰도 표시
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                cv2.putText(vis_img1, f'Disappeared ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체의 마스크 표시 (초록색)
        for obj in appeared:
            mask = obj['mask'].astype(bool)
            vis_img2[mask] = vis_img2[mask] * 0.7 + np.array([0, 255, 0]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img2, contours, -1, (0, 255, 0), 2)
            
            # 신뢰도 표시
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                cv2.putText(vis_img2, f'Appeared ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 변화 히트맵 생성
        change_heatmap = change_score.cpu().numpy()
        change_heatmap = (change_heatmap * 255).astype(np.uint8)
        change_heatmap = cv2.applyColorMap(change_heatmap, cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, change_heatmap, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, change_heatmap, alpha, 0)
        
        result = np.hstack((overlay1, overlay2))
        return result
    
    # 시각화 실행
    result_image = create_visualization(image1, image2, disappeared, appeared, change_score)
    
    # 결과 출력
    cv2.imshow('Object Changes (Segmentation)', result_image)
    
    print("\nChange Detection Results:")
    print(f"Total objects in first image: {len(masks1)}")
    print(f"Total objects in second image: {len(masks2)}")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            print(f"Class: {obj['class']}, Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 1.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 2.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    disappeared, appeared = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25,
        iou_threshold=0.5
    )


0: 640x640 3 crossarms, 1 polo, 9 wires, 19.3ms
Speed: 2.0ms preprocess, 19.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 crossarms, 1 polo, 12 wires, 16.3ms
Speed: 2.0ms preprocess, 16.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Total objects in first image: 4
Total objects in second image: 4
Disappeared objects: 0
Appeared objects: 0


In [41]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Conv3DNet(nn.Module):
    def __init__(self, in_channels=3):
        super(Conv3DNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(in_channels, 32, kernel_size=(2, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            
            nn.Conv3d(32, 16, kernel_size=(1, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(16),
            
            nn.Conv3d(16, 1, kernel_size=(1, 1, 1)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(2)
        return x

def analyze_image_difference(image1, image2):
    """두 이미지 간의 실제 차이 분석"""
    # 그레이스케일 변환
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    # 이미지 차분
    diff = cv2.absdiff(gray1, gray2)
    
    # 노이즈 제거
    diff = cv2.GaussianBlur(diff, (5, 5), 0)
    
    # 임계값 처리
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    return thresh

def match_masks(mask1, mask2, diff_mask):
    """두 마스크 간의 매칭 여부 판단"""
    # 마스크 영역에서의 차이 계산
    overlap = np.logical_and(mask1, mask2)
    diff_region = np.logical_and(diff_mask, np.logical_or(mask1, mask2))
    
    # 변화율 계산
    total_area = np.sum(mask1) + np.sum(mask2)
    if total_area == 0:
        return False
        
    change_ratio = np.sum(diff_region) / total_area * 2
    
    # 오버랩 비율 계산
    min_area = min(np.sum(mask1), np.sum(mask2))
    if min_area == 0:
        return False
        
    overlap_ratio = np.sum(overlap) / min_area
    
    # 매칭 판단
    return change_ratio < 0.3 and overlap_ratio > 0.5

def detect_changes(results1, results2, diff_mask, conf_threshold=0.25):
    """변화 감지"""
    disappeared = []
    appeared = []
    
    # 첫 번째 이미지의 객체들 처리
    matched1 = set()
    for i in range(len(results1.boxes)):
        if results1.boxes.conf[i].item() < conf_threshold:
            continue
            
        cls1 = int(results1.boxes.cls[i].item())
        mask1 = results1.masks.data[i].cpu().numpy()
        matched = False
        
        # 두 번째 이미지의 같은 클래스 객체들과 비교
        for j in range(len(results2.boxes)):
            if results2.boxes.conf[j].item() < conf_threshold:
                continue
                
            cls2 = int(results2.boxes.cls[j].item())
            if cls1 != cls2:
                continue
                
            mask2 = results2.masks.data[j].cpu().numpy()
            if match_masks(mask1, mask2, diff_mask):
                matched = True
                matched1.add(i)
                break
        
        if not matched:
            disappeared.append({
                'mask': mask1,
                'conf': results1.boxes.conf[i].item(),
                'class': cls1
            })
    
    # 두 번째 이미지의 새로운 객체들 처리
    for j in range(len(results2.boxes)):
        if results2.boxes.conf[j].item() < conf_threshold:
            continue
            
        cls2 = int(results2.boxes.cls[j].item())
        mask2 = results2.masks.data[j].cpu().numpy()
        matched = False
        
        # 첫 번째 이미지의 같은 클래스 객체들과 비교
        for i in range(len(results1.boxes)):
            if results1.boxes.conf[i].item() < conf_threshold:
                continue
                
            cls1 = int(results1.boxes.cls[i].item())
            if cls1 != cls2:
                continue
                
            mask1 = results1.masks.data[i].cpu().numpy()
            if match_masks(mask1, mask2, diff_mask):
                matched = True
                break
        
        if not matched:
            appeared.append({
                'mask': mask2,
                'conf': results2.boxes.conf[j].item(),
                'class': cls2
            })
    
    return disappeared, appeared

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25):
    # YOLO 모델 로드
    yolo_model = YOLO(yolo_model_path)
    
    # 3D CNN 모델 초기화
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    temporal_model = Conv3DNet().to(device)
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # 이미지 차분 분석
    diff_mask = analyze_image_difference(image1, image2)
    
    # YOLO로 객체 검출 및 세그멘테이션
    results1 = yolo_model(image1)[0]
    results2 = yolo_model(image2)[0]
    
    # 변화 감지
    disappeared, appeared = detect_changes(results1, results2, diff_mask, conf_threshold)
    
    # 3D CNN을 위한 입력 준비
    imgs_tensor = torch.from_numpy(np.stack([image1, image2]))
    imgs_tensor = imgs_tensor.permute(3, 0, 1, 2).unsqueeze(0).float() / 255.0
    imgs_tensor = imgs_tensor.to(device)
    
    # 3D CNN으로 변화 감지
    with torch.no_grad():
        change_score = temporal_model(imgs_tensor)
        change_score = change_score.squeeze()
    
    # 클래스 이름 매핑
    class_names = {
        0: 'crossarm',
        1: 'pole',
        2: 'wire'
    }
    
    # 결과 시각화
    def create_visualization(image1, image2, disappeared, appeared, change_score, diff_mask):
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 차분 결과 시각화 (선택적)
        diff_vis = cv2.cvtColor(diff_mask, cv2.COLOR_GRAY2BGR)
        
        # 사라진 객체의 마스크 표시 (빨간색)
        for obj in disappeared:
            mask = obj['mask'].astype(bool)
            vis_img1[mask] = vis_img1[mask] * 0.7 + np.array([0, 0, 255]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img1, contours, -1, (0, 0, 255), 2)
            
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
                cv2.putText(vis_img1, f'Disappeared {class_name} ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체의 마스크 표시 (초록색)
        for obj in appeared:
            mask = obj['mask'].astype(bool)
            vis_img2[mask] = vis_img2[mask] * 0.7 + np.array([0, 255, 0]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img2, contours, -1, (0, 255, 0), 2)
            
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
                cv2.putText(vis_img2, f'Appeared {class_name} ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 변화 히트맵 생성
        change_heatmap = change_score.cpu().numpy()
        change_heatmap = (change_heatmap * 255).astype(np.uint8)
        change_heatmap = cv2.applyColorMap(change_heatmap, cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, change_heatmap, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, change_heatmap, alpha, 0)
        
        result = np.hstack((overlay1, overlay2))
        return result
    
    # 시각화 실행
    result_image = create_visualization(image1, image2, disappeared, appeared, 
                                      change_score, diff_mask)
    
    # 결과 출력
    cv2.imshow('Object Changes (Difference-based)', result_image)
    
    print("\nChange Detection Results:")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
            print(f"{class_name}: Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
            print(f"{class_name}: Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

if __name__ == "__main__":
    image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 3.jpg'
    image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 4.jpg'
    yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
    
    disappeared, appeared = detect_object_changes(
        image1_path, 
        image2_path, 
        yolo_model_path,
        conf_threshold=0.25
    )


0: 640x640 1 crossarm, 1 polo, 13 wires, 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crossarm, 1 polo, 15 wires, 19.9ms
Speed: 2.0ms preprocess, 19.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Disappeared objects: 0
Appeared objects: 2

Appeared objects details:
wire: Confidence: 0.94
wire: Confidence: 0.93


In [58]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from ultralytics import YOLO

class Conv3DNet(nn.Module):
    def __init__(self, in_channels=3):
        super(Conv3DNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(in_channels, 32, kernel_size=(2, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            
            nn.Conv3d(32, 16, kernel_size=(1, 3, 3), padding=(0, 1, 1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(16),
            
            nn.Conv3d(16, 1, kernel_size=(1, 1, 1)),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(2)
        return x

def verify_wire_match(mask1, mask2, group_info=None):
    """개선된 wire 매칭 검증"""
    y1, x1 = np.where(mask1)
    y2, x2 = np.where(mask2)
    
    if len(x1) < 2 or len(x2) < 2:
        return False, 0.0
    
    try:
        # 기울기 계산
        coef1 = np.polyfit(x1, y1, 1)
        coef2 = np.polyfit(x2, y2, 1)
        slope1, slope2 = coef1[0], coef2[0]
        
        # 중심점 계산
        center1 = (np.mean(x1), np.mean(y1))
        center2 = (np.mean(x2), np.mean(y2))
        
        # 수직 거리 계산
        vertical_dist = abs(center1[1] - center2[1])
        
        # 기울기 차이
        slope_diff = abs(slope1 - slope2)
        
        # 평행선 그룹 검사
        if group_info is not None:
            group1 = group_info.get(center1[1], [])
            group2 = group_info.get(center2[1], [])
            if group1 and group2:
                avg_spacing1 = np.mean(np.diff(sorted([y for y, _ in group1])))
                avg_spacing2 = np.mean(np.diff(sorted([y for y, _ in group2])))
                spacing_match = abs(avg_spacing1 - avg_spacing2) < 10
            else:
                spacing_match = True
        else:
            spacing_match = True
        
        # 매칭 조건 강화
        matched = (
            vertical_dist < 15 and    # 수직 거리 15픽셀 이내
            slope_diff < 0.1 and      # 기울기 차이 0.1 이내
            spacing_match             # 평행선 간격 유사
        )
        
        score = 1.0 - (vertical_dist / 30 + slope_diff)
        return matched, max(0, min(1, score))
        
    except:
        return False, 0.0

def verify_crossarm_match(mask1, mask2):
    """crossarm 매칭을 위한 특별 검증"""
    y1, x1 = np.where(mask1)
    y2, x2 = np.where(mask2)
    
    if len(x1) < 2 or len(x2) < 2:
        return False, 0.0
    
    # 중심점 계산
    center1 = (np.mean(x1), np.mean(y1))
    center2 = (np.mean(x2), np.mean(y2))
    
    # 거리 계산
    center_dist = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
    
    # crossarm의 경우 위치가 거의 동일해야 함
    return center_dist < 15, 1.0 - (center_dist / 15)

def analyze_image_difference(image1, image2):
    """이미지 차분 분석"""
    # 이미지 전처리
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    # CLAHE 적용
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray1 = clahe.apply(gray1)
    gray2 = clahe.apply(gray2)
    
    # 노이즈 제거
    gray1 = cv2.GaussianBlur(gray1, (5, 5), 0)
    gray2 = cv2.GaussianBlur(gray2, (5, 5), 0)
    
    # 이미지 차분
    diff = cv2.absdiff(gray1, gray2)
    
    # 적응형 임계값 처리
    diff = cv2.adaptiveThreshold(
        diff,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11,
        2
    )
    
    # 노이즈 제거를 위한 모폴로지 연산
    kernel = np.ones((3,3), np.uint8)
    diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN, kernel)
    
    return diff

def match_masks(mask1, mask2, diff_mask):
    """일반적인 마스크 매칭"""
    overlap = np.logical_and(mask1, mask2)
    diff_region = np.logical_and(diff_mask, np.logical_or(mask1, mask2))
    
    total_area = np.sum(mask1) + np.sum(mask2)
    if total_area == 0:
        return False, 0.0
    
    min_area = min(np.sum(mask1), np.sum(mask2))
    overlap_ratio = np.sum(overlap) / min_area
    change_ratio = np.sum(diff_region) / total_area * 2
    
    # 완화된 매칭 조건
    matched = (change_ratio < 0.3 and overlap_ratio > 0.6)
    score = overlap_ratio * (1 - change_ratio)
    
    return matched, score

def detect_changes(results1, results2, diff_mask, conf_threshold=0.25):
    """변화 감지 함수 수정"""
    disappeared = []
    appeared = []
    matched_pairs = set()
    
    # wire 객체만 추출
    wires1 = [(i, results1.masks.data[i]) for i in range(len(results1.boxes)) 
             if int(results1.boxes.cls[i].item()) == 2 
             and results1.boxes.conf[i].item() > conf_threshold]
    
    wires2 = [(i, results2.masks.data[i]) for i in range(len(results2.boxes)) 
             if int(results2.boxes.cls[i].item()) == 2 
             and results2.boxes.conf[i].item() > conf_threshold]
    
    # wire 그룹화
    def group_wires(wires):
        groups = {}
        for idx, mask in wires:
            y_center = np.mean(np.where(mask.cpu().numpy())[0])
            groups.setdefault(int(y_center // 20) * 20, []).append((y_center, idx))
        return groups
    
    wire_groups1 = group_wires(wires1)
    wire_groups2 = group_wires(wires2)
    
    # wire 매칭
    for idx1, mask1 in wires1:
        if idx1 in [p[0] for p in matched_pairs]:
            continue
            
        best_match = -1
        best_score = -1
        
        for idx2, mask2 in wires2:
            if idx2 in [p[1] for p in matched_pairs]:
                continue
                
            matched, score = verify_wire_match(
                mask1.cpu().numpy(), 
                mask2.cpu().numpy(),
                group_info=wire_groups2
            )
            
            if matched and score > best_score:
                best_score = score
                best_match = idx2
        
        if best_match >= 0:
            matched_pairs.add((idx1, best_match))
        else:
            disappeared.append({
                'mask': mask1.cpu().numpy(),
                'conf': results1.boxes.conf[idx1].item(),
                'class': 2
            })
    
    # 매칭되지 않은 wire 처리
    for idx2, mask2 in wires2:
        if idx2 not in [p[1] for p in matched_pairs]:
            appeared.append({
                'mask': mask2.cpu().numpy(),
                'conf': results2.boxes.conf[idx2].item(),
                'class': 2
            })
    
    return disappeared, appeared

def detect_object_changes(image1_path, image2_path, yolo_model_path, conf_threshold=0.25):
    """메인 실행 함수"""
    # YOLO 모델 로드
    yolo_model = YOLO(yolo_model_path)
    
    # 3D CNN 모델 초기화
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    temporal_model = Conv3DNet().to(device)
    
    # 이미지 로드 및 전처리
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    # 이미지 크기 조정
    target_size = (640, 640)
    image1 = cv2.resize(image1, target_size)
    image2 = cv2.resize(image2, target_size)
    
    # 이미지 차분 분석
    diff_mask = analyze_image_difference(image1, image2)
    
    # YOLO로 객체 검출
    results1 = yolo_model(image1)[0]
    results2 = yolo_model(image2)[0]
    
    # 변화 감지
    disappeared, appeared = detect_changes(results1, results2, diff_mask, conf_threshold)
    
    # 3D CNN 처리
    imgs_tensor = torch.from_numpy(np.stack([image1, image2]))
    imgs_tensor = imgs_tensor.permute(3, 0, 1, 2).unsqueeze(0).float() / 255.0
    imgs_tensor = imgs_tensor.to(device)
    
    with torch.no_grad():
        change_score = temporal_model(imgs_tensor)
        change_score = change_score.squeeze()
    
    # 시각화 및 결과 출력
    class_names = {0: 'crossarm', 1: 'pole', 2: 'wire'}
    
    def create_visualization(image1, image2, disappeared, appeared, change_score):
        vis_img1 = image1.copy()
        vis_img2 = image2.copy()
        
        # 사라진 객체 표시
        for obj in disappeared:
            mask = obj['mask'].astype(bool)
            vis_img1[mask] = vis_img1[mask] * 0.7 + np.array([0, 0, 255]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img1, contours, -1, (0, 0, 255), 2)
            
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
                cv2.putText(vis_img1, f'Disappeared {class_name} ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # 새로 나타난 객체 표시
        for obj in appeared:
            mask = obj['mask'].astype(bool)
            vis_img2[mask] = vis_img2[mask] * 0.7 + np.array([0, 255, 0]) * 0.3
            
            contours, _ = cv2.findContours((mask * 255).astype(np.uint8), 
                                         cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(vis_img2, contours, -1, (0, 255, 0), 2)
            
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
                cv2.putText(vis_img2, f'Appeared {class_name} ({obj["conf"]:.2f})', 
                           (cx-60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 히트맵 처리
        change_heatmap = change_score.cpu().numpy()
        change_heatmap = (change_heatmap * 255).astype(np.uint8)
        change_heatmap = cv2.applyColorMap(change_heatmap, cv2.COLORMAP_JET)
        
        # 결과 합치기
        alpha = 0.3
        overlay1 = cv2.addWeighted(vis_img1, 1 - alpha, change_heatmap, alpha, 0)
        overlay2 = cv2.addWeighted(vis_img2, 1 - alpha, change_heatmap, alpha, 0)
        
        return np.hstack((overlay1, overlay2))
    
    # 시각화 실행
    result_image = create_visualization(image1, image2, disappeared, appeared, change_score)
    cv2.imshow('Object Changes', result_image)
    
    # 결과 출력
    print("\nChange Detection Results:")
    print(f"Disappeared objects: {len(disappeared)}")
    print(f"Appeared objects: {len(appeared)}")
    
    if disappeared:
        print("\nDisappeared objects details:")
        for obj in disappeared:
            class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
            print(f"{class_name}: Confidence: {obj['conf']:.2f}")
    
    if appeared:
        print("\nAppeared objects details:")
        for obj in appeared:
            class_name = class_names.get(obj['class'], f'class_{obj["class"]}')
            print(f"{class_name}: Confidence: {obj['conf']:.2f}")
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return disappeared, appeared

if __name__ == "__main__":
   image1_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 5.jpg'
   image2_path = 'C:/Users/AI-LHJ/Downloads/source/ch5/test 6.jpg'
   yolo_model_path = 'C:/Users/AI-LHJ/Desktop/YOLOv11/runs/segment/train11/weights/best.pt'
   
   disappeared, appeared = detect_object_changes(
       image1_path, 
       image2_path, 
       yolo_model_path,
       conf_threshold=0.25
   )


0: 640x640 4 crossarms, 1 polo, 15 wires, 19.9ms
Speed: 2.0ms preprocess, 19.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 crossarms, 1 polo, 19 wires, 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

Change Detection Results:
Disappeared objects: 1
Appeared objects: 5

Disappeared objects details:
wire: Confidence: 0.80

Appeared objects details:
wire: Confidence: 0.95
wire: Confidence: 0.94
wire: Confidence: 0.91
wire: Confidence: 0.91
wire: Confidence: 0.86
